In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import gzip
import json

In [2]:
import string

In [3]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk import FreqDist

In [4]:
import numpy as np
import gensim
from gensim import corpora

In [115]:
import contractions

In [118]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Musical_Instruments_5.json.gz')

In [320]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [120]:
data = df[['reviewText']]

In [122]:
data['no_contr']= data['reviewText'].apply(lambda x: contractions.fix(x))

In [124]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
data['clean_msg']= data['no_contr'].apply(lambda x:remove_punctuation(x))

In [125]:
data['msg_lower']= data['clean_msg'].apply(lambda x: x.lower())

In [126]:
def remove_stopwords(text):
    tokens = word_tokenize(text)
    output= [i for i in tokens if i not in stopwords]
    return output
data['no_stopwords']= data['clean_msg'].apply(lambda x:remove_stopwords(x))

In [127]:
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
data['msg_lemmatized']=data['no_stopwords'].apply(lambda x:lemmatizer(x))

In [129]:
data['finalText'] = [' '.join(map(str, l)) for l in data['msg_lemmatized']]

In [130]:
data = data.sample(frac=1).reset_index(drop=True)[:1000]

In [319]:
data.head()

,reviewText,no_contr,clean_msg,msg_lower,no_stopwords,msg_lemmatized,finalText
0,I received this yesterday and it is the best t...,I received this yesterday and it is the best t...,I received this yesterday and it is the best t...,i received this yesterday and it is the best t...,"[I, received, yesterday, best, tuner, acoustic...","[I, received, yesterday, best, tuner, acoustic...",I received yesterday best tuner acoustic guita...
1,I just put it on my Baby Taylor guitar strap. ...,I just put it on my Baby Taylor guitar strap. ...,I just put it on my Baby Taylor guitar strap ...,i just put it on my baby taylor guitar strap ...,"[I, put, Baby, Taylor, guitar, strap, Nice, Do...","[I, put, Baby, Taylor, guitar, strap, Nice, Do...",I put Baby Taylor guitar strap Nice Does job w...
2,"Once you setup your pedalboard... that's it, y...","Once you setup your pedalboard... that is it, ...",Once you setup your pedalboard that is it you ...,once you setup your pedalboard that is it you ...,"[Once, setup, pedalboard, going, move, stretch...","[Once, setup, pedalboard, going, move, stretch...",Once setup pedalboard going move stretch step ...
3,"I used to use a Schubb Capo, but started using...","I used to use a Schubb Capo, but started using...",I used to use a Schubb Capo but started using ...,i used to use a schubb capo but started using ...,"[I, used, use, Schubb, Capo, started, using, D...","[I, used, use, Schubb, Capo, started, using, D...",I used use Schubb Capo started using Dunlop Tr...
4,I have tried nearly every kind of popular stri...,I have tried nearly every kind of popular stri...,I have tried nearly every kind of popular stri...,i have tried nearly every kind of popular stri...,"[I, tried, nearly, every, kind, popular, strin...","[I, tried, nearly, every, kind, popular, strin...",I tried nearly every kind popular string I use...


Мой подход: извлечь наиболее часто встречающиеся в тексте существительные, перед эти сделав препроцессинг (конечно же). идея в том, что скорее всего эти существительные - ключевые слова, т.е. упоминания продуктов в отзывах. минус - останутся лишние, но тоже часто упоминающиеся существительные как price, sound.

Была идея использовать ворд2век для title или features, но открыв метаданные, поняла что они есть у малой части товаров, что не очень понятно почему. скорее всего, я плохо искала, но json падал два раза, когда я открывала файл с метаданными, что усложнило работу.

Можно было бы также использовать нейросеть для извлечения NE (например, БЕРТовскую модель)

Я взяла корпус музыкальных инструментов, но поняла, что корпус не просто для привычных нам инструментов, как гитара, пианино, скрипка и т.д., а для различных устройств типа педалей, частей типа струн и т.д., что, как мне кажется, тоже важно.

### буду пытаться извлечь упоминания с помощью spacy

In [131]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

In [132]:
tokens = nlp(''.join(str(data.finalText.tolist())))

In [133]:
from collections import Counter

In [134]:
items = [x.text for x in tokens.ents]
Counter(items).most_common(20)

[('one', 167),
 ('two', 89),
 ('first', 54),
 ('Amazon', 37),
 ('2', 37),
 ('three', 31),
 ('5', 28),
 ('3', 27),
 ('4', 26),
 ('jack', 23),
 ('second', 22),
 ('12', 21),
 ('100', 19),
 ('20', 18),
 ('DAddario', 15),
 ('EQ', 15),
 ('6', 15),
 ('XLR', 15),
 ('10', 15),
 ('Dunlop', 14)]

In [135]:
product_list = []
for ent in tokens.ents:
    if ent.label_ == 'PRODUCT':
        product_list.append(ent.text)
        
product_counts = Counter(product_list).most_common(20)
df_product = pd.DataFrame(product_counts, columns =['text', 'count'])

In [136]:
df_product

,text,count
0,TS808,5
1,Behringer,5
2,DP008,3
3,TO800,3
4,Dunlop 5000 5005,2
5,DP004,2
6,GT100,2
7,US100,2
8,EtoE,1
9,Electrified,1


тэг PRODUCT выдает названия моделей инструментов, а не сами инструменты

In [137]:
org_list = []
for ent in tokens.ents:
    if ent.label_ == 'ORG':
        org_list.append(ent.text)
        
org_counts = Counter(org_list).most_common(20)
df_org = pd.DataFrame(org_counts, columns =['text', 'count'])

In [138]:
df_org

,text,count
0,Amazon,37
1,XLR,15
2,AC,14
3,USB,10
4,Dunlop,9
5,MXR,9
6,FAB,6
7,Samson,6
8,iPad,5
9,Yamaha,5


тэг ORG как и ожидалось выдал бренды

In [322]:
def get_kw(x):
    doc=nlp(''.join(str(x.tolist()))) 
    doc=[i.text for i in doc if i.text not in stopwords and i.pos_=="NOUN"] 
    doc=list(map(lambda i: i.lower(),doc)) 
    doc=pd.Series(doc)
    doc=doc.value_counts().head().index.tolist()
    return doc

In [323]:
print(get_kw(data.finalText))

['guitar', 'string', 'pedal', 'sound', 'price']


ура

In [144]:
from spacy.lang.en import English

In [145]:
import textacy
parser = English()

### извлечем n-граммы

In [146]:
doc = textacy.make_spacy_doc((''.join(str(data.finalText.tolist()))) , 'en_core_web_sm')

In [147]:
bigrams = list(textacy.extract.basics.ngrams(doc, 2, min_freq=2))

In [149]:
str_bigrams = [i.text for i in bigrams]

In [150]:
Counter(str_bigrams).most_common(10)

[('sound great', 34),
 ('work great', 34),
 ('power supply', 32),
 ('Planet Waves', 32),
 ('acoustic guitar', 30),
 ('pedal board', 28),
 ('sound good', 26),
 ('long time', 24),
 ('tube amp', 23),
 ('highly recommend', 23)]

In [151]:
trigrams = list(textacy.extract.basics.ngrams(doc, 3, min_freq=2))

In [152]:
str_trigrams = [i.text for i in trigrams]

In [153]:
Counter(str_trigrams).most_common(10)

[('local music store', 7),
 ('solid state amp', 7),
 ('string I tried', 5),
 ('Little Big Muff', 5),
 ('built like tank', 5),
 ('Fender Super Champ', 5),
 ('acoustic electric guitar', 4),
 ('guitar sound like', 4),
 ('BOSS GT10 Patches', 4),
 ('GT10 Patches James', 4)]

In [267]:
y = [word.split() for word in str_bigrams]

In [268]:
bigr = [tuple(l) for l in y]

In [300]:
z = [word.split() for word in str_trigrams]

In [301]:
trigr = [tuple(l) for l in z]

### метрики

In [226]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [287]:
finder2 = BigramCollocationFinder.from_documents(bigr)
finder3 = TrigramCollocationFinder.from_words(str_trigrams)

In [296]:
finder2.apply_freq_filter(3)
finder3.apply_freq_filter(2)

In [292]:
finder2.score_ngrams(bigram_measures.pmi)[:15]

[(('Cyber', 'Twin'), 12.109504215757006),
 (('Shure', 'SM58'), 12.109504215757006),
 (('customer', 'service'), 12.109504215757006),
 (('exceeds', 'expectation'), 12.109504215757006),
 (('strain', 'relief'), 12.109504215757006),
 (('auto', 'punch'), 11.694466716478162),
 (('completely', 'satisfied'), 11.694466716478162),
 (('regular', 'basis'), 11.694466716478162),
 (('Bad', 'Monkey'), 11.69446671647816),
 (('Schubb', 'S1'), 11.69446671647816),
 (('form', 'factor'), 11.69446671647816),
 (('5000', '5005'), 11.372538621590799),
 (('Custom', 'Comp'), 11.372538621590799),
 (('fairly', 'thick'), 11.372538621590799),
 (('phosphor', 'bronze'), 11.372538621590799)]

In [293]:
finder2.score_ngrams(bigram_measures.student_t)[:15]

[(('Planet', 'Waves'), 5.637812451130488),
 (('power', 'supply'), 5.625277934001512),
 (('work', 'great'), 5.2869998311753745),
 (('pedal', 'board'), 5.105841865680807),
 (('acoustic', 'guitar'), 5.039072318482129),
 (('long', 'time'), 4.850754636590992),
 (('highly', 'recommend'), 4.772490240182245),
 (('tube', 'amp'), 4.6655984880564345),
 (('work', 'fine'), 4.6116776812779765),
 (('easy', 'use'), 4.526329849815411),
 (('Les', 'Paul'), 4.461610900930334),
 (('sound', 'great'), 4.369080723732374),
 (('Ernie', 'Ball'), 4.234106547797429),
 (('electric', 'guitar'), 4.216821638906677),
 (('year', 'ago'), 4.214549145184843)]

In [299]:
finder2.score_ngrams(bigram_measures.chi_sq)[:15]

[(('Bad', 'Monkey'), 13257.0),
 (('Cyber', 'Twin'), 13257.0),
 (('Schubb', 'S1'), 13257.0),
 (('Shure', 'SM58'), 13257.0),
 (('customer', 'service'), 13257.0),
 (('exceeds', 'expectation'), 13257.0),
 (('form', 'factor'), 13257.0),
 (('strain', 'relief'), 13257.0),
 (('planet', 'wave'), 11046.666352248718),
 (('HD', '500'), 9954.00970884542),
 (('auto', 'punch'), 9941.999830239927),
 (('completely', 'satisfied'), 9941.999830239927),
 (('regular', 'basis'), 9941.999830239927),
 (('Planet', 'Waves'), 9496.89471163931),
 (('pop', 'filter'), 9467.856603854943)]

Сложно сказать, какая метрика лучше подходит для отчета. PMI хуже стьюдента и хи-квадрата, стьюдент ранжирует выше название инструмента, тогда как хи-квадрат - название определенной модели инструмента (хи-квадрат детальнее?) 

In [303]:
finder3.score_ngrams(trigram_measures.pmi)[:10]

[(('BOSS GT10 guitar', 'GT10 guitar multi', 'guitar multi effect'),
  16.860905103331064),
 (('Eddie Van Halen', 'Van Halen Joe', 'Halen Joe Satriani'),
  16.860905103331064),
 (('GT10 guitar multi', 'guitar multi effect', 'multi effect processor'),
  16.860905103331064),
 (('Halen Joe Satriani',
   'Joe Satriani guitarist',
   'Satriani guitarist studio'),
  16.860905103331064),
 (('James Limborg Search', 'Limborg Search Google', 'Search Google BOSS'),
  16.860905103331064),
 (('Joe Satriani guitarist',
   'Satriani guitarist studio',
   'guitarist studio album'),
  16.860905103331064),
 (('KS8291XX Pro Double', 'Pro Double X', 'Double X Keyboard'),
  16.860905103331064),
 (('Limborg Search Google', 'Search Google BOSS', 'Google BOSS GT10'),
  16.860905103331064),
 (('Pro Double X', 'Double X Keyboard', 'height attachment bar'),
  16.860905103331064),
 (('Satriani guitarist studio',
   'guitarist studio album',
   'studio album guitar'),
  16.860905103331064)]

In [304]:
finder3.score_ngrams(trigram_measures.student_t)[:10]

[(('BOSS GT10 Patches', 'GT10 Patches James', 'Patches James Limborg'),
  1.9999327872295736),
 (('JoYo pedal built', 'pedal built extremely', 'extremely well perform'),
  1.732018065587638),
 (('pedal built extremely', 'extremely well perform', 'perform good better'),
  1.732018065587638),
 (('purchased several JoYo', 'JoYo pedal built', 'pedal built extremely'),
  1.732018065587638),
 (('sold I purchased', 'purchased several JoYo', 'JoYo pedal built'),
  1.732018065587638),
 (('BOSS GT10 guitar', 'GT10 guitar multi', 'guitar multi effect'),
  1.4142016807216573),
 (('Eddie Van Halen', 'Van Halen Joe', 'Halen Joe Satriani'),
  1.4142016807216573),
 (('GT10 guitar multi', 'guitar multi effect', 'multi effect processor'),
  1.4142016807216573),
 (('Halen Joe Satriani',
   'Joe Satriani guitarist',
   'Satriani guitarist studio'),
  1.4142016807216573),
 (('James Limborg Search', 'Limborg Search Google', 'Search Google BOSS'),
  1.4142016807216573)]

In [305]:
finder3.score_ngrams(trigram_measures.chi_sq)[:10]

[(('BOSS GT10 guitar', 'GT10 guitar multi', 'guitar multi effect'),
  238052.00581395338),
 (('Eddie Van Halen', 'Van Halen Joe', 'Halen Joe Satriani'),
  238052.00581395338),
 (('GT10 guitar multi', 'guitar multi effect', 'multi effect processor'),
  238052.00581395338),
 (('Halen Joe Satriani',
   'Joe Satriani guitarist',
   'Satriani guitarist studio'),
  238052.00581395338),
 (('James Limborg Search', 'Limborg Search Google', 'Search Google BOSS'),
  238052.00581395338),
 (('Joe Satriani guitarist',
   'Satriani guitarist studio',
   'guitarist studio album'),
  238052.00581395338),
 (('KS8291XX Pro Double', 'Pro Double X', 'Double X Keyboard'),
  238052.00581395338),
 (('Limborg Search Google', 'Search Google BOSS', 'Google BOSS GT10'),
  238052.00581395338),
 (('Pro Double X', 'Double X Keyboard', 'height attachment bar'),
  238052.00581395338),
 (('Satriani guitarist studio',
   'guitarist studio album',
   'studio album guitar'),
  238052.00581395338)]

метрики триграмм выглядят очень похоже друг на друга, сложно выбрать одно

### выведем группы по 5

In [216]:
mus_words = ['guitar', 'string', 'pedal']
dic_group = {}
for i in mus_words:
    group = []
    for j in Counter(str_bigrams):
        if i in j:
            group.append(j)
    dic_group[i] = group

In [312]:
print(list(dic_group)[0])
print('-----')
print('\n'.join(dic_group.get('guitar')[:5]))

guitar
-----
acoustic guitar
guitar player
Taylor guitar
guitar strap
fender guitar


In [315]:
print(list(dic_group)[1])
print('-----')
print('\n'.join(dic_group.get('string')[:5]))

string
-----
fender string
needed string
kind string
GREAT string
pressure string


In [316]:
print(list(dic_group)[2])
print('-----')
print('\n'.join(dic_group.get('pedal')[:5]))

pedal
-----
foot pedal
Distortion pedal
pedal sort
use pedal
pedal board
